<a href="https://colab.research.google.com/github/rahakaushik/quant_finance/blob/main/CMO_Bollinger_Entry_Exit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

In [3]:
# Function: Chande Momentum Oscillator (CMO)
def chande_momentum_oscillator(df, period):
    df['Up'] = np.where(df['Close'] > df['Close'].shift(1), df['Close'] - df['Close'].shift(1), 0)
    df['Down'] = np.where(df['Close'] < df['Close'].shift(1), df['Close'].shift(1) - df['Close'], 0)
    df['SumUp'] = df['Up'].rolling(window=period).sum()
    df['SumDown'] = df['Down'].rolling(window=period).sum()
    df['CMO'] = 100 * (df['SumUp'] - df['SumDown']) / (df['SumUp'] + df['SumDown'])
    return df

In [4]:
# Function: Calculate Bollinger Bands
def calculate_bollinger_bands(df, period, std_dev):
    df['Middle_Band'] = df['Close'].rolling(window=period).mean()
    rolling_std = df['Close'].rolling(window=period).std()
    df['Upper_Band'] = df['Middle_Band'] + (rolling_std * std_dev)
    df['Lower_Band'] = df['Middle_Band'] - (rolling_std * std_dev)
    return df

In [5]:
# Download the data
symbol = 'TSLA'
start_date = '2023-01-01'
end_date = '2025-03-20'
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)  # Handle missing data


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [6]:
# Parameter ranges for optimization
cmo_period_range = range(1, 51)  # CMO period
bollinger_period_range = range(1, 51)  # Bollinger Bands period
std_dev_range = np.arange(1.5, 3.0, 0.5)  # Standard deviation: 1.5, 2.0, 2.5

In [7]:
# Optimization
best_params = None
best_performance = -np.inf

for cmo_period, bb_period, std_dev in itertools.product(cmo_period_range, bollinger_period_range, std_dev_range):
    temp_df = df.copy()

    # Calculate indicators
    temp_df = chande_momentum_oscillator(temp_df, period=cmo_period)
    temp_df = calculate_bollinger_bands(temp_df, period=bb_period, std_dev=std_dev)

    # Define Entry and Exit Signalstemp_df['Entry'] = (
    temp_df['Entry'] = (
        (temp_df['CMO'] < -50) &
        (temp_df['Close'] > temp_df['Lower_Band'])
    )
    temp_df['Exit'] = (
        (temp_df['CMO'] > 50) &
        (temp_df['Close'] < temp_df['Upper_Band'])
    )# Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=temp_df['Close'],
        entries=temp_df['Entry'].to_numpy(),
        exits=temp_df['Exit'].to_numpy(),
        init_cash=100_000,
        fees=0.001
          )
    # Evaluate performance (Total Return as an example)
    performance = portfolio.total_return()

    if performance > best_performance:
        best_performance = performance
        best_params = (cmo_period, bb_period, std_dev)

In [8]:
print(f"Best Parameters: CMO Period = {best_params[0]}, Bollinger Period = {best_params[1]}, Std Dev = {best_params[2]}")
print(f"Best Performance (Total Return): {best_performance}")
best_cmo_period  = best_params[0]
best_bb_period = best_params[1]
best_std_dev = best_params[2]

Best Parameters: CMO Period = 18, Bollinger Period = 27, Std Dev = 2.0
Best Performance (Total Return): 2.190269185983799


In [9]:
# Calculate indicators: CMO and Bollinger Bands
df = chande_momentum_oscillator(df, period=best_cmo_period)
df = calculate_bollinger_bands(df, period=best_bb_period, std_dev=best_std_dev)

# Define Entry and Exit Signals
df['Entry'] = (
    (df['CMO'] < -50) &  # CMO is below -50
    (df['Close'] > df['Lower_Band'])  # Price closes above lower Bollinger Band
)

df['Exit'] = (
    (df['CMO'] > 50) &  # CMO is above +50
    (df['Close'] < df['Upper_Band'])  # Price closes below upper Bollinger Band
)

# Convert signals to boolean arrays
entries = df['Entry'].to_numpy()
exits = df['Exit'].to_numpy()

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2023-01-03 00:00:00
End                           2025-03-19 00:00:00
Period                                        554
Start Value                              100000.0
End Value                           319026.918598
Total Return [%]                       219.026919
Benchmark Return [%]                   118.186868
Max Gross Exposure [%]                      100.0
Total Fees Paid                       1585.354447
Max Drawdown [%]                        45.666309
Max Drawdown Duration                       131.0
Total Trades                                    4
Total Closed Trades                             3
Total Open Trades                               1
Open Trade PnL                      -66378.920489
Win Rate [%]                                100.0
Best Trade [%]                          95.398206
Worst Trade [%]                         23.372301
Avg Winning Trade [%]                   59.604532
Avg Losing Trade [%]                          NaN
